In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from random import randint
from time import sleep
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [ ]:
auth_file="secrets.txt"

playlist_ids=["3LgwHwHUnheGm88n8pDdA7","2JHheevZhehi18oX4t4lXE","1hMzceeWw7QiI6vaBkcEJO",
"4rnleEAOdmFAbRcNCgZMpY","69fEt9DN5r4JQATi52sRtq","5S8SJdl1BDc0ugpkEvFsIL","7ikZLGjkIILujRBn2GL5bM",
"37i9dQZF1DX4PP3DA4J0N8","0vvXsWCC9xrXsKd4FyS8kM","4rnleEAOdmFAbRcNCgZMpY","2YRe7HRKNRvXdJBp9nXFza",
"5xqpyfZyS1DVysoevdVyEn","6yPiKpy7evrwvZodByKvM9","61GYfverId8JaqpXnvy0kb","3fjVK89rcsV4TomUaxYt15"]

In [ ]:
def init_spotify(auth_file):
    secrets_file = open(auth_file,"r")
    secrets_dict={}
    for line in secrets_file.read().split('\n'):
        if len(line) > 0:
            secrets_dict[line.split(':')[0]]=line.split(':')[1]
    return spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=secrets_dict['clientid'],
                                                               client_secret=secrets_dict['clientsecret']))


In [ ]:
sp=init_spotify(auth_file)

In [ ]:
def get_playlist_tracks(playlist_id):
    results = sp.user_playlist_tracks("spotify",playlist_id)
    tracks = results['items']
    cnt=1
    while results['next']!=None:
        print("Retrieving tracks from Page ",cnt,"..")
        results = sp.next(results)
        tracks = tracks + results['items']
        cnt+=1
        sleep(randint(1,3))
    return tracks

In [ ]:
def get_name_artists_href_uri_from_track(track):
    return [(track["name"],artist["name"],track["href"],track["uri"]) for artist in track["artists"]]


In [ ]:
def flatten(input_list):
    return [item for sublist in input_list for item in sublist]

In [ ]:
def get_name_artists_href_uri_from_list(tracks):
    return flatten(list(map(get_name_artists_href_uri_from_track,tracks['items'])))

In [ ]:
def get_name_artist_href_uri_from_playlist_item(playlist_item):
    return get_name_artists_href_uri_from_track(playlist_item['track'])

In [ ]:
def get_name_artists_href_uri_from_playlist(input_playlist):
    return flatten(list(map(get_name_artist_href_uri_from_playlist_item,input_playlist["items"])))

In [ ]:
def get_df_from_all_tracks(all_tracks,save=False):
    dict_spotify_info_lst=[]
    dict_spotify_audio_feat_lst=[]
    progress_cnt=0
    print("Processed: ",progress_cnt/len(all_tracks),"%")
    for track in all_tracks:
        dict_spotify_info={}
        dict_spotify_info['track'],dict_spotify_info['artist'],dict_spotify_info['href'],dict_spotify_info['uri']=get_name_artist_href_uri_from_playlist_item(track)[0]
        dict_spotify_audio_feat=sp.audio_features(track['track']['uri'])[0]

        if dict_spotify_audio_feat is not None:
            dict_spotify_info_lst.append(dict_spotify_info)
            dict_spotify_audio_feat_lst.append(dict_spotify_audio_feat)
        progress_cnt+=1
        if (progress_cnt/len(all_tracks))%10==0:
            print("Processed: ",progress_cnt/len(all_tracks),"%")

    df_spotify_info = pd.DataFrame.from_dict(dict_spotify_info_lst)
    df_spotify_audio_feat = pd.DataFrame.from_dict(dict_spotify_audio_feat_lst)
    
    if save:
        df_spotify_info.to_csv("output_files/df_spotify_info.csv",index=False)
        df_spotify_audio_feat.to_csv("output_files/df_spotify_audio_feat.csv",index=False)
        
    return df_spotify_info,df_spotify_audio_feat

In [ ]:
#GET TRACK NAME AND ARTIST

In [ ]:
all_playlist_tracks=[]

for playlist_id in playlist_ids:
    playlist_tracks = get_playlist_tracks(playlist_id)
    all_playlist_tracks = all_playlist_tracks + playlist_tracks

In [ ]:
#Get Track Info and Audio Features of the Playlist Tracks into DataFrames

In [ ]:
df_spotify_info,df_spotify_audio_feat=get_df_from_all_tracks(all_playlist_tracks,save=True)

In [ ]:
#REMOVING DUPLICATES

In [ ]:
df_spotify_info=df_spotify_info.drop_duplicates(subset='uri')
df_spotify_audio_feat=df_spotify_audio_feat.drop_duplicates(subset='uri')

In [ ]:
df_spotify_info_copy.to_csv("output_files/df_spotify_info.csv",index=False)
df_spotify_audio_feat_copy.to_csv("output_files/df_spotify_audio_feat.csv",index=False)